# Set Up

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Notebook variables - inherit from pipeline or job
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

# Customer

In [0]:
df_customer_bronze_pyspark = (spark.read
                              .format("json")
                              .option("inferSchema", "true")
                              .load("/Volumes/workspace/sim_retail_demo/customer")
                              .select(
                                  "*",
                                  F.current_timestamp().alias("processing_time"),
                                  "_metadata.file_name"
                              )
)

df_customer_bronze_pyspark.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.customer_bronze_pyspark")

# Orders

In [0]:
df_orders_bronze_pyspark = (spark.read
                              .format("json")
                              .option("inferSchema", "true")
                              .load("/Volumes/workspace/sim_retail_demo/orders")
                              .select(
                                  "*",
                                  F.current_timestamp().alias("processing_time"),
                                  "_metadata.file_name"
                              )
)

df_orders_bronze_pyspark.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.orders_bronze_pyspark")

# Status

In [0]:
df_status_bronze_pyspark = (spark.read
                              .format("json")
                              .option("inferSchema", "true")
                              .load("/Volumes/workspace/sim_retail_demo/status")
                              .select(
                                  "*",
                                  F.current_timestamp().alias("processing_time"),
                                  "_metadata.file_name"
                              )
)

df_status_bronze_pyspark.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.status_bronze_pyspark")